<a href="https://colab.research.google.com/github/Sanarazaaa/RNA-seq-Based-Machine-Learning-for-PTSD-Classification/blob/main/RNA_seq_Based_Machine_Learning_for_PTSD_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

expr_file = "GSE97356_RawCounts.csv"
meta_file = "metadata.csv"

expr = pd.read_csv(expr_file, index_col=0)
meta = pd.read_csv(meta_file)

print(f"Expression shape (genes x samples): {expr.shape}")
print(f"Metadata shape (rows): {meta.shape}")

if "Sample Name" not in meta.columns or "ptsd" not in meta.columns:
    print("WARNING: 'Sample Name' and/or 'ptsd' missing in metadata; cannot proceed.")
else:
    n = min(expr.shape[1], meta.shape[0])
    if expr.shape[1] != meta.shape[0]:
        print(f"NOTE: Mismatched counts. Using first {n} samples in both files (order-based alignment).")
    expr = expr.iloc[:, :n].copy()
    meta = meta.iloc[:n].copy()

    expr.columns = meta["Sample Name"].astype(str).values

    labels_raw = meta["ptsd"].astype(str).str.strip().str.title()
    label_map = {"Never": 0, "Past": 1, "Current": 1}
    y = labels_raw.map(label_map)

    valid = ~y.isna()
    if valid.sum() < 2:
        print("WARNING: Too few labeled samples after cleaning. Cannot train a classifier.")
    else:
        dropped = (~valid).sum()
        if dropped > 0:
            print(f"NOTE: Dropping {dropped} samples with missing/unmapped labels.")
        expr = expr.loc[:, valid.values]
        y = y[valid].astype(int)

        X_counts = expr.copy()
        min_samples = max(1, int(0.2 * X_counts.shape[1]))
        keep = (X_counts >= 10).sum(axis=1) >= min_samples
        kept_genes = int(keep.sum())
        if kept_genes == 0:
            print("WARNING: Low-expression filter removed all genes; skipping filter.")
            X_counts_f = X_counts
        else:
            print(f"Kept {kept_genes} genes after low-expression filtering "
                  f"(>=10 counts in >={min_samples} samples).")
            X_counts_f = X_counts.loc[keep]

        X_log = np.log2(X_counts_f + 1.0)
        X = X_log.T.values
        y = y.values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        def counts(arr):
            zeros = np.sum(arr == 0)
            ones  = np.sum(arr == 1)
            return f"[0: {zeros}, 1: {ones}]"
        print("Class distribution (full):", counts(y))
        print("Class distribution (train):", counts(y_train))
        print("Class distribution (test):", counts(y_test))

        k = min(1000, X.shape[1])
        pipeline = Pipeline([
            ("select", SelectKBest(score_func=f_classif, k=k)),
            ("scale", StandardScaler(with_mean=True)),
            ("svm", SVC(kernel="linear", class_weight="balanced"))
        ])

        param_grid = {"svm__C": [0.01, 0.1, 1, 10]}

        cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid = GridSearchCV(
            estimator=pipeline,
            param_grid=param_grid,
            scoring="f1",
            cv=cv_inner,
            n_jobs=-1,
            verbose=1
        )

        grid.fit(X_train, y_train)
        print("Best parameters:", grid.best_params_)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))

        print("\nClassification Report:")
        print(classification_report(y_test, y_pred, digits=3))

        cv_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        scoring = {
            "accuracy": "accuracy",
            "precision": "precision_macro",
            "recall": "recall_macro",
            "f1": "f1_macro",
            "roc_auc": "roc_auc"
        }
        try:
            cv_results = cross_validate(best_model, X, y, cv=cv_outer, scoring=scoring, n_jobs=-1)
            print("\n5-fold cross-validation (using best pipeline):")
            for m in scoring:
                vals = cv_results[f"test_{m}"]
                print(f"{m}: {vals.mean():.3f} (+/- {vals.std():.3f})")
        except Exception as e:
            print("\nCross-validation warning:", str(e))
            print("Continuing without CV scores that failed.")


Expression shape (genes x samples): (25830, 324)
Metadata shape (rows): (324, 28)
Kept 16231 genes after low-expression filtering (>=10 counts in >=64 samples).
Class distribution (full): [0: 201, 1: 123]
Class distribution (train): [0: 161, 1: 98]
Class distribution (test): [0: 40, 1: 25]
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters: {'svm__C': 0.01}

Confusion Matrix:
[[24 16]
 [ 7 18]]

Classification Report:
              precision    recall  f1-score   support

           0      0.774     0.600     0.676        40
           1      0.529     0.720     0.610        25

    accuracy                          0.646        65
   macro avg      0.652     0.660     0.643        65
weighted avg      0.680     0.646     0.651        65


5-fold cross-validation (using best pipeline):
accuracy: 0.543 (+/- 0.043)
precision: 0.527 (+/- 0.048)
recall: 0.528 (+/- 0.051)
f1: 0.525 (+/- 0.047)
roc_auc: 0.550 (+/- 0.085)
